BiLSTM序列标注---盲汉翻译

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

https://blog.csdn.net/vivian_ll/article/details/93894151

In [14]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # LSTM以word_embeddings作为输入, 输出维度为 hidden_dim 的隐藏状态值
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # 线性层将隐藏状态空间映射到标注空间
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # 一开始并没有隐藏状态所以我们要先初始化一个
        # 关于维度为什么这么设计请参考Pytoch相关文档
        # 各个维度的含义是 (num_layers*num_directions, batch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [15]:
#Making training data
training_data = []
t = open('./对应拼音标调.txt',"r", encoding='UTF8')
f = open("./训练集.txt", "rb")
#计数 数一共有多少行
lines = f.readlines()
tags = t.readlines()

# Training data: 6692(total number of sentences)*2(line&tag)
for index, line in enumerate(lines):
    line = lines[index].rstrip().split()
    tag = tags[index].rstrip().split()
    training_data.append((line, tag))

In [16]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"0":0, "1":1, "2":2, "3":3, "4":4, ",":5, ".":6, "?":7,"!":8}

np.save('word_to_ix.npy',word_to_ix)
np.save('tag_to_ix.npy',tag_to_ix)

In [29]:
# Training the LSTM

#Monitor trianing results
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

#Helper function to convert data into tensor
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# Accuracy Recording
acc_record = []

for i in range(10):
    #Change parameters to reach better performances
    EMBEDDING_DIM = 60 
    HIDDEN_DIM = 10 + 20*i
    g = "./log/"+"Hidden_dim_"+str(EMBEDDING_DIM)+"_Embedding_dim_"+str(HIDDEN_DIM)
    model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
    loss_function = nn.NLLLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    writer = SummaryWriter(g)


    # # No training, so use no_grad()
    # with torch.no_grad():
    #     inputs = prepare_sequence(training_data[0][0], word_to_ix)
    #     tag_scores = model(inputs)
    #     targets = prepare_sequence(training_data[0][1], tag_to_ix)
    #     #print((torch.argmax(tag_scores, dim=-1) == targets).sum().item() / len(targets))

    EPOCH=30

    for epoch in range(EPOCH):  
        print(epoch)
        training_acuracy = []
        validation_acuracy = []
        loss_list_train = []
        loss_list_validation = []
        for sentence, tags in tqdm(training_data[:5000]):
            # 第一步: 请记住Pytorch会累加梯度.
            # 我们需要在训练每个实例前清空梯度
            model.zero_grad()

            # 此外还需要清空 LSTM 的隐状态,
            # 将其从上个实例的历史中分离出来.
            model.hidden = model.init_hidden()

            # 准备网络输入, 将其变为词索引的 Tensor 类型数据            
            sentence_in = prepare_sequence(sentence, word_to_ix)
            targets = prepare_sequence(tags, tag_to_ix)
            
            # 第三步: 前向传播.
            tag_scores = model(sentence_in)

            # 第四步: 计算损失和梯度值, 通过调用 optimizer.step() 来更新梯度
            loss = loss_function(tag_scores, targets)
            a = (torch.argmax(tag_scores, dim=-1) == targets).sum().item() / len(targets)
            training_acuracy.append(a)
            loss.backward()
            optimizer.step()
            loss_list_train.append(loss.item())
        #Validation
        
        for sentence, tags in tqdm(training_data[5000:6000]):
            model.zero_grad()
            model.hidden = model.init_hidden()

            # 准备网络输入, 将其变为词索引的 Tensor 类型数据
            
            sentence_in = prepare_sequence(sentence, word_to_ix)
            targets = prepare_sequence(tags, tag_to_ix)
            # 第三步: 前向传播.
            tag_scores = model(sentence_in)
            #计算损失
            a = (torch.argmax(tag_scores, dim=-1) == targets).sum().item() / len(targets)
            validation_acuracy.append(a)
            loss = loss_function(tag_scores, targets)
            loss_list_validation.append(loss.item())
        
        acc_record.append((a,HIDDEN_DIM))
            
        #LSTM 可以做padding
        
        print(loss)
        c= sum(loss_list_train)
        writer.add_scalar('Loss/training', c, epoch)
        d = sum(loss_list_validation)
        writer.add_scalar('Loss/validation', d, epoch)
        #print(torch.argmax(tag_scores, dim=-1))
        print("training_acuracy",sum(training_acuracy)/len(training_acuracy))
        b = sum(training_acuracy)/len(training_acuracy)
        writer.add_scalar('Accuracy/training', b, epoch)
        print("validation_acuracy:",sum(validation_acuracy)/len(validation_acuracy))
        a =sum(validation_acuracy)/len(validation_acuracy) 
        writer.add_scalar('Accuracy/validation', a, epoch)

    # 查看训练后的得分
    with torch.no_grad():
        inputs = prepare_sequence(training_data[0][0], word_to_ix)
        tag_scores = model(inputs)
writer.close()


0


100%|██████████| 1000/1000 [00:02<00:00, 389.52it/s]


tensor(0.4989, grad_fn=<NllLossBackward0>)
training_acuracy 0.6593967083002259
validation_acuracy: 0.7170687534080743
1


100%|██████████| 1000/1000 [00:02<00:00, 406.65it/s]


tensor(0.4255, grad_fn=<NllLossBackward0>)
training_acuracy 0.7572296516277447
validation_acuracy: 0.7340861134985377
2


...

tensor(0.0052, grad_fn=<NllLossBackward0>)
training_acuracy 0.9886495160884784
validation_acuracy: 0.9452604846506931
28


100%|██████████| 1000/1000 [00:02<00:00, 357.06it/s]


tensor(0.0049, grad_fn=<NllLossBackward0>)
training_acuracy 0.9886465134429756
validation_acuracy: 0.9451741534380461
29


100%|██████████| 1000/1000 [00:02<00:00, 356.10it/s]


tensor(0.0047, grad_fn=<NllLossBackward0>)
training_acuracy 0.9887175313956114
validation_acuracy: 0.9452618658563468
